In [1]:
%autoawait

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
import time
import asyncio

In [3]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8052
WEBHOOK_BASE = "/"
ADMIN_URL = "http://researcher-agent:8051"

In [4]:
#Paste in invitation from data_owner agent
invite = {'connection_id': '95e91f5d-3b56-4d36-9a74-ee17568d7b17', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '5dde53f0-fd0e-4467-813f-a29da9e3994f', 'recipientKeys': ['8WaYDnM4YYP8ECakf5dTVNbZQuvHTNgVeuZRosHru8Uq'], 'serviceEndpoint': 'http://192.168.65.3:8020', 'label': 'Bob'}, 'invitation_url': 'http://192.168.65.3:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNWRkZTUzZjAtZmQwZS00NDY3LTgxM2YtYTI5ZGE5ZTM5OTRmIiwgInJlY2lwaWVudEtleXMiOiBbIjhXYVlEbk00WVlQOEVDYWtmNWRUVk5iWlF1dkhUTmdWZXVaUm9zSHJ1OFVxIl0sICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovLzE5Mi4xNjguNjUuMzo4MDIwIiwgImxhYmVsIjogIkJvYiJ9'}

In [5]:
async def start_agent():
    # Based on the aca-py agent you wish to control
    agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                           webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)
    
    def research_messages_hook(payload):
        connection_id = payload["connection_id"]
        print("Handle research messages ", payload, connection_id)

    research_message_listener = {
        "handler": research_messages_hook,
        "topic": "basicmessages"
    }
    
#     def data_messages_hook(payload):
#         connection_id = payload["connection_id"]
#         print("Handle data messages ", payload, connection_id)


#     data_message_listener = {
#         "handler": data_messages_hook,
#         "topic": "basicmessages"
#     }
    
    
    await agent_controller.listen_webhooks()
    
    # Controllers re-emit webhooks as events through PyPubSub
    # Basic state management is handled through defaults but you can choose to pass in custom handlers
    agent_controller.register_listeners([research_message_listener], defaults=True)
    
    # Receive Invitation
    response = await agent_controller.connections.accept_connection(invite["invitation"])
    print(response)
    
    # Print out accepted Invite and Researcher ID
    print("Researcher ID", response["connection_id"])
    researcher_id = response["connection_id"]
    print("Invite Accepted")
    print(response)
    
    await asyncio.sleep(30)
    # Do a trust ping to ensure connection is accepted by data_owner
    trust_ping = await agent_controller.connections.trust_ping(researcher_id,"hello")
    print("TUST PING TO ACTIVATE CONNECTION - RESEARCH -> DATA")
    print(trust_ping)    
    
    # Print connection list
    connection = await agent_controller.connections.get_connection(researcher_id)
    print("RESEARCH AGENT CONNECTION")
    print(connection)

    #send some basic messages
    message = await agent_controller.connections.send_message(researcher_id,"hello from researcher world!")
    print("BASIC MESSAGE - RESEARCH -> DATA")
    print(message)

In [6]:
async def main():
    await asyncio.gather(start_agent())

s = time.perf_counter()
await main()
elapsed = time.perf_counter() - s
print(f"Script executed in {elapsed:0.2f} seconds.")

ACCEPTED CONNECTION -  {'updated_at': '2020-06-24 21:31:16.418515Z', 'invitation_key': '8WaYDnM4YYP8ECakf5dTVNbZQuvHTNgVeuZRosHru8Uq', 'state': 'request', 'created_at': '2020-06-24 21:31:16.387151Z', 'accept': 'manual', 'initiator': 'external', 'routing_state': 'none', 'invitation_mode': 'once', 'connection_id': '1fd25eda-ae38-48b7-b094-fc0f526dd4a4', 'their_label': 'Bob', 'request_id': '43d8e342-12d5-4aea-9380-b0a91812c16f', 'my_did': 'UwKxBXxdvWG8pqBzqBDZTy'}
{'updated_at': '2020-06-24 21:31:16.418515Z', 'invitation_key': '8WaYDnM4YYP8ECakf5dTVNbZQuvHTNgVeuZRosHru8Uq', 'state': 'request', 'created_at': '2020-06-24 21:31:16.387151Z', 'accept': 'manual', 'initiator': 'external', 'routing_state': 'none', 'invitation_mode': 'once', 'connection_id': '1fd25eda-ae38-48b7-b094-fc0f526dd4a4', 'their_label': 'Bob', 'request_id': '43d8e342-12d5-4aea-9380-b0a91812c16f', 'my_did': 'UwKxBXxdvWG8pqBzqBDZTy'}
Researcher ID 1fd25eda-ae38-48b7-b094-fc0f526dd4a4
Invite Accepted
{'updated_at': '2020-06-

In [ ]:
# if __name__ == "__main__":

#     try:
#         asyncio.get_event_loop().run_until_complete(start_agent())
#     except KeyboardInterrupt:
#         os._exit(1)